# Yakondi stats and visualizations
Based on data export from May 2, 2020 (i.e. stale data!)

Also note that other scripts exist in https://gitlab.com/mikebochenek/jupyter-notebooks

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import pymysql.cursors
import json

def fetchData(sql):
    connection = pymysql.connect(host='localhost', user='test2', password='test2', db='beta', cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql)
            rows = cursor.fetchall()
    finally:
        connection.close()
    return rows

In [8]:
display(pd.DataFrame(fetchData("select role,count(*) as c from Users group by role")))

,role,c
0,adviser,421
1,blocked,3
2,end-user,1429
3,user,1


In [10]:
d = pd.DataFrame(fetchData("""select count(*) as total, t.tag_name as country from Tags t 
    join TagReferences tr on tr.tag_id = t.tag_id and t.status = 3
    join Users u on u.user_id = tr.ref_id and u.role = 'ADVISER'
    group by t.tag_name order by total desc""" ))
display(d.head(20))

,total,country
0,225,italy
1,217,france
2,210,spain
3,200,germany
4,162,United States
5,151,netherlands
6,149,thailand
7,146,United Kingdom
8,123,austria
9,116,greece


In [16]:
df = pd.DataFrame(fetchData("""select user_id,currentLocation,misc from Users where role = 'adviser' 
  order by user_id desc"""))  # convert rows from fetch into data frame

df = df.dropna(axis=0, how='any')
# print ('df size ' + str(len(df)))

l = []
for index, row in df.iterrows():
    j = json.loads(row['misc'])
    l.append(j['countryCode'])

from collections import Counter

# df.count()

x = Counter(l)    
countries = sorted(x.items(), key=lambda i: i[1], reverse=True)
print (countries)
# countries - looks prettier

[('us', 46), ('in', 35), ('gb', 30), ('ca', 23), ('ch', 23), ('de', 20), ('es', 15), ('it', 13), ('au', 12), ('id', 10), ('lv', 10), ('nl', 10), ('ph', 9), ('fr', 8), ('th', 7), ('dk', 7), (None, 6), ('my', 6), ('at', 6), ('', 5), ('lk', 5), ('ro', 5), ('vn', 5), ('pt', 5), ('ee', 5), ('ae', 5), ('mx', 5), ('tr', 4), ('gr', 4), ('be', 4), ('co', 4), ('no', 4), ('za', 4), ('sg', 3), ('ar', 3), ('br', 3), ('ie', 3), ('jp', 3), ('cl', 3), ('tz', 3), ('sn', 2), ('si', 2), ('ma', 2), ('hn', 2), ('il', 2), ('mu', 2), ('rs', 2), ('nz', 2), ('ke', 2), ('ir', 2), ('is', 1), ('hr', 1), ('np', 1), ('eg', 1), ('kh', 1), ('im', 1), ('tg', 1), ('mt', 1), ('pl', 1), ('ci', 1), ('lt', 1), ('pk', 1), ('bn', 1), ('zm', 1), ('mg', 1), ('cz', 1), ('hk', 1), ('ba', 1), ('mv', 1), ('cm', 1)]
